In [12]:
import sys
print(sys.executable)


/home/floriann/.conda/envs/eubucco-features/bin/python


In [11]:
gpd.GeoDataFrame(geometry=[], crs=3035).to_crs(3035)

,geometry


In [13]:
import sys
import os

import h3pandas
import geopandas as gpd
import pandas as pd
import numpy as np
import h3
import osmnx as ox

# # project lib
# PROJECT_SRC_PATH = os.path.join(os.path.abspath(''), '..')
# sys.path.append(PROJECT_SRC_PATH)



In [14]:
%load_ext autoreload
%autoreload 2

In [ ]:
pd.set_option("display.max_columns", None)

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)

In [133]:
for file in only_in_dir2:
    if 'gpkg' in file:
        print(file)

## Overlay Gov, OSM, and  MSFT

In [ ]:
gdf_geojson.explore()

In [ ]:
nuts[nuts['NUTS_ID'] == 'CZ072'].explore()

In [ ]:
nuts[nuts[nuts['NUTS_ID'] == 'CH032'].to_crs(gdf_geojson.crs).intersects(gdf_geojson)]

In [37]:
gov = gpd.read_parquet('/p/projects/eubucco/data/3-attrib-cleaning-v1-gov/FRK26.parquet').to_crs(3035)
osm = gpd.read_parquet('/p/projects/eubucco/data/3-attrib-cleaning-v1-osm/FRK26.parquet').to_crs(3035)
msft = gpd.read_file('/p/projects/eubucco/data/2-database-nuts-level-v1-msft/france/FRK26.gpkg').to_crs(3035)

# gov = gpd.read_file('../overlay-test-data/gov-FRK26.gpkg').to_crs(3035)
# osm = gpd.read_file('../overlay-test-data/osm-FRK26.gpkg').to_crs(3035)
# msft = gpd.read_file('../overlay-test-data/msft-FRK26.gpkg').to_crs(3035)

gov['dataset'] = 'gov'
osm['dataset'] = 'osm'
msft['dataset'] = 'msft'

In [38]:
# msft['source_file'] = msft['source_file'].astype(str)
# gov_plus_osm_msft.to_parquet('/p/tmp/floriann/test/test.parquet')
msft.to_parquet('/p/tmp/floriann/test/msft.parquet')

# msft.dtypes

In [21]:
some_lau_ids = [
    # 'FR69123',
    # 'FR69202',
    'FR69089',
    'FR69043',
    'FR69204',
    'FR69149',
    # different region below
    # 'FR69001',
]

In [33]:
gov_sample = gov[gov['LAU_ID'].isin(some_lau_ids)]
osm_sample = osm[osm['LAU_ID'].isin(some_lau_ids)]
msft_sample = msft[msft['LAU_ID'].isin(some_lau_ids)]


In [17]:
from shapely.geometry import Polygon
from shapely.strtree import STRtree

def filter_non_intersecting(base_gdf, target_gdf):
    tree = STRtree(base_gdf.geometry)
    non_intersecting = target_gdf[~target_gdf.geometry.apply(lambda geom: any(tree.query(geom)))] # based on bouding box!
    return non_intersecting

def calculate_iou(geom1, geom2):
    if not geom1.intersects(geom2):
        return 0
    intersection = geom1.intersection(geom2).area
    union = geom1.union(geom2).area
    return intersection / union

def calculate_ioa(geom1, geom2):
    if not geom1.intersects(geom2):
        return 0
    intersection = geom1.intersection(geom2).area
    area = min(geom1.area, geom2.area)
    return intersection / area

def passes_ioa_threshold(geom, potential_matches, ioa_threshold):
    if potential_matches.empty:
        return True, True

    ioas = []
    for match in potential_matches:
        ioa = calculate_ioa(geom, match)

        if ioa >= ioa_threshold:
            return False, False  # Abort early if IoA exceeds threshold
        
        ioas.append(ioa)

    if max(ioas) == 0:
        return True, True

    return False, True

def filter_by_ioa(base_gdf, target_gdf, ioa_threshold=0.2):
    tree = STRtree(base_gdf.geometry)
    valid_rows = []

    for _, row in target_gdf.iterrows():
        geom = row.geometry
        potential_matches = tree.query(geom)
        # print(potential_matches)
        # print(type(potential_matches))
        # max_ioa = max((calculate_ioa(geom, ) for match in potential_matches), default=0)
        # if max_ioa < ioa_threshold:
        non_itersecting, intersecting_below_threshold = passes_ioa_threshold(geom, base_gdf.geometry.iloc[potential_matches], ioa_threshold)
        row['non_intersecting'] = non_itersecting

        if intersecting_below_threshold:
            valid_rows.append(row)

    return gpd.GeoDataFrame(valid_rows, crs=target_gdf.crs)



def gpd_concat(*gdfs):
    """
    Concatenate multiple GeoDataFrames, ensuring they share the same CRS.
    
    Parameters:
        *gdfs: One or more GeoDataFrames to concatenate.
        
    Returns:
        gpd.GeoDataFrame: The concatenated GeoDataFrame with a unified CRS.
    """
    if not gdfs:
        raise ValueError("At least one GeoDataFrame must be provided.")

    base_crs = gdfs[0].crs
    reprojected_gdfs = [gdf.to_crs(base_crs) if gdf.crs != base_crs else gdf for gdf in gdfs]
    
    return gpd.GeoDataFrame(pd.concat(reprojected_gdfs), crs=base_crs)


# Process datasets

# filtered_osm = filter_non_intersecting(gov, osm)
# gov_plus_osm = gpd_concat(gov, filtered_osm)
# filtered_msft = filter_non_intersecting(gov_plus_osm, msft)
# result_gdf_full = gpd_concat(gov_plus_osm, filtered_msft)

# filtered_osm = filter_non_intersecting(gov_sample, osm_sample)
# gov_plus_osm = gpd_concat(gov_sample, filtered_osm)
# filtered_msft = filter_non_intersecting(gov_plus_osm, msft_sample)
# result_gdf = gpd_concat(gov_plus_osm, filtered_msft)

# filtered_osm = filter_by_ioa(gov_sample, osm_sample)
# gov_plus_osm = gpd_concat(gov_sample, filtered_osm)
# filtered_msft = filter_by_ioa(gov_plus_osm, msft_sample)
# result_gdf_ioa_2 = gpd_concat(gov_plus_osm, filtered_msft)


# Save or visualize the result
# result_gdf.to_file("filtered_result.shp")
# result_gdf.explore('dataset')

In [ ]:
# def snearest(left: gpd.GeoDataFrame, right: gpd.GeoDataFrame, max_distance: float = None) -> pd.DataFrame:
#     exclusive = left is right
#     (left_i, right_i), dis = right.sindex.nearest(
#         left.geometry, return_all=False, return_distance=True, max_distance=max_distance, exclusive=exclusive
#     )

#     nearest = right.iloc[right_i].reset_index()
#     nearest.index = left.index[left_i]
#     nearest["distance"] = dis

#     return nearest



# def calculate_ioa(gdf1, gdf2):
#     if not gdf1.intersects(gdf2):
#         return 0
#     intersection = gdf1.intersects(gdf2).area
#     area = min(gdf1.area, geom2.area)
#     return intersection / area



# nearest = snearest(gov_sample, osm_sample, max_distance=5)
# # calculate_ioa(gov_sample.loc[nearest.index], nearest.geometry)
# gdf1 = gov_sample.loc[nearest.index]
# gdf2 = nearest.geometry
(osm_sample_i, gov_sample_i), dis = gov_sample.sindex.nearest(
    osm_sample.geometry, return_all=False, return_distance=True, max_distance=1, exclusive=False
)



# detail with return_all=True
# max distance=0 not possible, is there a better function?
gdf1 = gov_sample.iloc[gov_sample_i]
gdf2 = osm_sample.iloc[osm_sample_i].geometry
intersection = gdf1.intersection(gdf2).area
ioa = intersection / np.minimum(gdf1.area, gdf2.area)
ioa.hist(bins=100)
# nearest
# osm_sample.loc[calculate_ioa['index']]


In [ ]:
np.minimum(gov_sample.area, osm_sample.area)

In [34]:

def other_indices(indices, df):
    return np.setdiff1d(np.arange(len(df)), indices)


def most_frequent_category(s):
    mode = s.groupby(level=0).apply(pd.Series.mode)
    single_mode = mode.groupby(level=0).first()  # alternatively: get mode of larger building
    return single_mode


def intersection_to_area_ratio(gdf1, gdf2): # or gpd.Series
    intersection = gdf1.intersection(gdf2).area
    area = np.minimum(gdf1.area, gdf2.area)
    return intersection / area


def merge_building_datasets(gdf1, gdf2, fillna):
    # determine intersecting buildings
    int_idx2, int_idx1 = gdf1.sindex.query(gdf2.geometry, predicate='intersects')

    # assess degree of overlap
    gdf1_int = gdf1.iloc[int_idx1]
    gdf2_int = gdf2.iloc[int_idx2]
    geoms1 = gdf1_int.geometry.reset_index()
    geoms2 = gdf2_int.geometry.reset_index()
    ioa = intersection_to_area_ratio(geoms1, geoms2)
    gdf2_int['ioa'] = ioa.values

    # add new non-intersecting buildings
    non_intersecting = gdf2.drop(gdf2_int.index)

    # add new slightly intersecting buildings
    gdf2_largest_int = gdf2_int.sort_values('ioa', ascending=False).drop_duplicates(keep='first')
    intersecting_below_thresh = gdf2_largest_int[gdf2_largest_int['ioa'] < 0.2]

    # use gdf2 to fill missing attributes in gdf1
    if fillna:
        gdf2_overlapping = gdf2_int[gdf2_int['ioa'] > 0.8]
        gdf2_overlapping.index = gdf1_int.index[ioa > 0.8]

        matched_type = most_frequent_category(gdf2_overlapping['type'])
        matched_height = gdf2_overlapping.groupby(level=0)['height'].mean()
        matched_age = gdf2_overlapping.groupby(level=0)['age'].mean()

        type_missings = gdf1['type'].isna()
        height_missings = gdf1['height'].isna()
        age_missings = gdf1['age'].isna()

        gdf1['type'] = gdf1['type'].fillna(matched_type)
        gdf1['height'] = gdf1['height'].fillna(matched_height)
        gdf1['age'] = gdf1['age'].fillna(matched_age)

        gdf1['filled_type'] = type_missings & gdf1['type'].notna()
        gdf1['filled_height'] = height_missings & gdf1['height'].notna()
        gdf1['filled_age'] = age_missings & gdf1['age'].notna()
        
    return pd.concat([gdf1, non_intersecting, intersecting_below_thresh])


# def merge_building_datasets(gdf1, gdf2, fillna):
#     # determine intersecting buildings
#     int_idx2, int_idx1 = gdf1.sindex.query(gdf2.geometry, predicate='intersects')

#     # assess degree of overlap
#     geoms1 = gdf1.geometry.iloc[int_idx1].reset_index()
#     geoms2 = gdf2.geometry.iloc[int_idx2].reset_index()
#     ioa = intersection_to_area_ratio(geoms1, geoms2)

#     # add new non-intersecting buildings
#     non_intersecting = gdf2.iloc[other_indices(int_idx2, gdf2)]

#     # add new slightly intersecting buildings
#     below_threshold = ioa < 0.2
#     intersecting_below_thresh = gdf2.iloc[int_idx2[below_threshold]]
#     intersecting_below_thresh['ioa'] = ioa[below_threshold].values

#     # use gdf2 to fill missing attributes in gdf1
#     if fillna:
#         above_threshold = ioa > 0.8
#         gdf2_overlapping = gdf2.iloc[int_idx2[above_threshold]]
#         gdf2_overlapping.index = gdf1.index[int_idx1[above_threshold]]

#         matched_type = most_frequent_category(gdf2_overlapping['type'])
#         matched_height = gdf2_overlapping.groupby(level=0)['height'].mean()
#         matched_age = gdf2_overlapping.groupby(level=0)['age'].mean()

#         gdf1['type'] = gdf1['type'].fillna(matched_type)
#         gdf1['height'] = gdf1['height'].fillna(matched_height)
#         gdf1['age'] = gdf1['age'].fillna(matched_age)

#     return pd.concat([gdf1, non_intersecting, intersecting_below_thresh])

gov_plus_osm = merge_building_datasets(gov_sample, osm_sample, fillna=True)
gov_plus_osm_msft = merge_building_datasets(gov_plus_osm, msft_sample, fillna=False)

/home/floriann/.conda/envs/eubucco-features/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/floriann/.conda/envs/eubucco-features/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/floriann/.conda/envs/eubucco-features/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is tryin

In [ ]:
# determine intersecting buildings
osm_int_indices, gov_int_indices = gov_sample.sindex.query(osm_sample.geometry, predicate='intersects')
osm_geoms = osm_sample.geometry.iloc[osm_int_indices].reset_index()
gov_geoms = gov_sample.geometry.iloc[gov_int_indices].reset_index()

# assess degree of overlap
ioa = intersection_to_area_ratio(gov_geoms, osm_geoms)

# add new non-intersecting buildings
non_intersecting = osm_sample.iloc[other_indices(osm_int_indices, osm_sample)]

# add new slightly intersecting buildings
below_threshold = ioa < 0.2
intersect_below_thresh = osm_sample.iloc[osm_int_indices[below_threshold]]
intersect_below_thresh['ioa'] = ioa[below_threshold].values

# join missing attributes
above_threshold = ioa > 0.8
overlapping_osm = osm_sample.iloc[osm_int_indices[above_threshold]]
overlapping_osm.index = gov_sample.index[gov_int_indices[above_threshold]]

matched_type = most_frequent_category(overlapping_osm['type'])
matched_height = overlapping_osm.groupby(level=0)['height'].mean()
matched_age = overlapping_osm.groupby(level=0)['age'].mean()

gov_sample['type'] = gov_sample['type'].fillna(matched_type)
gov_sample['height'] = gov_sample['height'].fillna(matched_height)
gov_sample['age'] = gov_sample['age'].fillna(matched_age)

pd.concat([gov_sample, non_intersecting, intersect_below_thresh])


# gov_sample['ioa_age'] could add column indicating degree of overlap when joining attributes to allow neglecting such attributes
# matched_type = pd.Series(overlapping_osm['type'].values, index=overlapping_gov.index)
# matched_type = most_frequent_category(matched_type)
# gov_sample['type'] = gov_sample['type'].fillna(matched_type)

# matched_height = pd.Series(overlapping_osm['height'].values, index=overlapping_gov.index)
# matched_height = most_frequent_category(matched_height)
# gov_sample['type'] = gov_sample['type'].fillna(matched_height)

# > 0.8 join attributes
# < 0.2 add new building (flag overlap violation)
# == 0 add new building

# most_freq_attr = high_overlap.groupby(level=0).apply(lambda x: x.mode(dropna=False).iloc[0])
# ioa.hist(bins=100)
# for osm_i, gov_i in zip(osm_int_indices, gov_indices):
#     calculate_ioa(osm_sample.iloc[osm_i].geometry, gov_sample.iloc[gov_i].geometry)


/home/floriann/.conda/envs/eubucco-features/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/floriann/.conda/envs/eubucco-features/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,id,LAU_ID,height,type_source,type,age,floors,source_file,Unnamed: 0,geometry,id_source,residential_type,height_source,dataset,ioa
78950,v1-FR69043-78950,FR69043,6.5,Commercial et services,commercial,1981.0,2.0,BATIMENT-auvergne-rhone-alpes,39423976.0,"POLYGON ((3911476.307 2525072.193, 3911468.776...",BATIMENT0000000235208874,NaN,6.5,gov,NaN
78951,v1-FR69043-78951,FR69043,7.5,Résidentiel,residential,1980.0,2.0,BATIMENT-auvergne-rhone-alpes,39423977.0,"POLYGON ((3911381.928 2525088.793, 3911385.503...",BATIMENT0000000235208875,NaN,7.5,gov,NaN
78952,v1-FR69043-78952,FR69043,3.2,Résidentiel,residential,1978.0,1.0,BATIMENT-auvergne-rhone-alpes,39423978.0,"POLYGON ((3911383.463 2524992.864, 3911381.409...",BATIMENT0000000235208877,NaN,3.2,gov,NaN
78953,v1-FR69043-78953,FR69043,5.5,Résidentiel,residential,1980.0,2.0,BATIMENT-auvergne-rhone-alpes,39423979.0,"POLYGON ((3911359.359 2525101.834, 3911363.116...",BATIMENT0000000235208879,NaN,5.5,gov,NaN
78954,v1-FR69043-78954,FR69043,4.1,Résidentiel,residential,1973.0,1.0,BATIMENT-auvergne-rhone-alpes,39423980.0,"POLYGON ((3911506.654 2524991.259, 3911514.574...",BATIMENT0000000235208880,NaN,4.1,gov,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402314,v1-FR69204-402314,FR69204,NaN,<NA>,NaN,NaN,NaN,rhone-alpes-latest,NaN,"POLYGON ((3915418.659 2523325.135, 3915421.638...",rhone-alpes-latest_3976486,NaN,floors,osm,0.098680
402317,v1-FR69204-402317,FR69204,NaN,church,public,NaN,NaN,rhone-alpes-latest,NaN,"POLYGON ((3915623.209 2523520.481, 3915624.959...",rhone-alpes-latest_1339021,NaN,floors,osm,0.052751
402318,v1-FR69204-402318,FR69204,NaN,church,public,NaN,NaN,rhone-alpes-latest,NaN,"POLYGON ((3915405.358 2523354.494, 3915423.471...",rhone-alpes-latest_1339023,NaN,floors,osm,0.035149
402318,v1-FR69204-402318,FR69204,NaN,church,public,NaN,NaN,rhone-alpes-latest,NaN,"POLYGON ((3915405.358 2523354.494, 3915423.471...",rhone-alpes-latest_1339023,NaN,floors,osm,0.009655


In [37]:
# gov_sample.iloc[gov_indices[:2]].explore()
# osm_sample.iloc[osm_indices[:2]]
# high_overlap.loc[82434]

# mode = high_overlap.groupby(level=0).apply(pd.Series.mode)
# mode
# mode[mode.index.get_level_values(1) == 1]

# single_mode = mode.groupby(level=0).first()

matched_height.dropna().head(50)
gov_indices[above_threshold]

array([ 3230,  3230,   155, ..., 22428, 22486, 22484], shape=(20483,))

In [52]:
df.index

NameError: name 'df' is not defined

In [58]:
gov_plus_osm_msft['dataset'].value_counts(normalize=True)


dataset
gov     0.584027
osm     0.348053
msft    0.067920
Name: proportion, dtype: float64

In [7]:
result_gdf_ioa_2['dataset'].value_counts(normalize=True)


dataset
gov     0.909943
osm     0.057215
msft    0.032842
Name: proportion, dtype: float64

In [ ]:
result_gdf['dataset'].value_counts(normalize=True)

In [ ]:
result_gdf_full['dataset'].value_counts(normalize=True)

result_gdf_full.groupby('LAU_ID')['dataset'].value_counts(normalize=True)


In [ ]:
gov_plus_osm_msft[gov_plus_osm_msft['LAU_ID'] == 'FR69043'].explore('dataset')


In [8]:
result_gdf_ioa2[result_gdf_ioa2['LAU_ID'].isin(some_lau_ids)].explore('dataset')

NameError: name 'result_gdf_ioa2' is not defined

In [ ]:
result_gdf_ioa[result_gdf_ioa['LAU_ID'].isin(some_lau_ids)].explore('dataset')

In [ ]:
result_gdf[result_gdf['LAU_ID'].isin(some_lau_ids)].explore('dataset')

In [ ]:
msft_sample.explore()